<a href="https://colab.research.google.com/github/DongIn-Kim/StockResearch/blob/main/stockdatabase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install pymysql

     |████████████████████████████████| 51kB 3.9MB/s 


In [3]:
import pymysql

class DBUpdater:
    def __init__(self):
      self.conn = pymysql.connent(host='localhost', user='root', password='SRquant21!', db='INVESTAR', charset='utf8')

      with self.conn.cursor() as curs:
        sql = """
        CREATE TABLE IF NOT EXISTS company_info (
          code VARCHAR(20),
          company VARCHAR(40),
	        last_update DATE,
	        PRIMARY KEY (CODE))
        """
        curs.execute(sql)
        sql = """
        CREATE TABLE IF NOT EXISTS daily_price (
          CODE VARCHAR(20),
          DATE DATE,
          OPEN BIGINT(20),
          high BIGINT(20),
          low BIGINT(20),
          close BIGINT(20),
          diff BIGINT(20),
          volume BIGINT(20),
          PRIMARY KEY (CODE,DATE))
        """

        curs.execute(sql)
      self.conn.commit()

      self.codes = dict()
      self.update_comp_info()()

    def __del__(self):
      """소멸자: MariaDB 연결 해제"""
      self.conn.close()

In [7]:
import pandas as pd
class DBUpdater :
    def read_krx_code(self):
      url = 'https://kind.krx.co.kr/corpgeneral/corpList.do?method=loadInitPage'
      krx = pd.read_heml(url, header=0)[0]
      krx = krx[['종목코드', '회사명']]
      krx = krx.rename(columns={'종목코드':'code','회사명':'company'})
      krx.code = krx.code.map('{:06d}'.format)
      return krx


In [8]:
import panddas as pd
from datetime import datetime

class DBUpdater:
  def update_comp_info(self):
    """종목코드를 company_info 테이블에 업데이트한 후 딕셔너리에 저장"""
    sql = "SELECT * FROM company_info"
    df = pd.read_sql(sql, self.conn)
    for idx in range(len(df)):
      self.codes[df['code'].values[idx]]=df['company'].values[idx]
    with self.conn.cursor() as curs:
      sql = "SELECT max(last_update) FROM company_info"
      curs.execute(sql)
      rs = curs.fetchone()
      today = datetime.today().strftime('%Y-%m-%d')

      if rs[0] == None or rs[0].strftime('%Y-%m-%d') < today:
        krx = self.read_krx_code()
        for idx in range(len(krx)):
          code = krx.code.values[idx]
          company = krx.company.values[idx]
          sql = f"REPLACE INTO company_info (code, company last"\f"update) VALUES ('{code}', '{company}', '{today}')"
          curs.execute(sql)
          self.codes[code] = company
          tmnow = datetime.now().strftime('%Y-%m-%d' %H:%M')
          print(f"[{tmnow}] {idx:04d} REPLACE INTO company_info "\f"VALUES ({code}, {company}, {today}")
        self.conn.commit()
        print(')

SyntaxError: ignored